# 朴素贝叶斯(Naive Bayes)文本分类算法——杜金鸿

## 一.定义
（1）设$x=\{a_1,\cdots,a_m\}$为一个待分类项，而每个$a_i$为$x$的一个特征属性;

（2）有类别集合$C=\{y_1,\cdots,y_n\}$;

（3）计算$P(y_1|x),\cdots,P(y_n|x)$;

（4）若$P(y_k|x)=\max\{P(y_1|x),\cdots,P(y_n|x)\}$，则$x\in y_k$.

## 二.条件概率的计算
（1）得到一个已知分类的待分类集合，即训练集；

（2）统计各类别下的各特征属性，用频率估计概率，得到条件概率估计，即$$P(a_1|y_1),P(a_2|y_1),\cdots,P(a_m|y_1)$$
$$P(a_1|y_2),P(a_2|y_2),\cdots,P(a_m|y_2)$$
$$.$$
$$.$$
$$.$$
$$P(a_1|y_n),P(a_2|y_n),\cdots,P(a_m|y_n);$$

（3）假设个特征属性相互独立，则它们条件独立，则根据贝叶斯公式有$$P(y_i|x)=\frac{P(x|y_i)P(y_i)}{P(x)}$$
    因为右端分母对于选定的$y_1,\cdots,y_n$为常数，只要将分子最大化即可，又有各特征属性相互独立，所以有$$P(x|y_i)P(y_i)=P(a_1|y_i)P(a_2|y_i)\cdots P(a_m|y_i)=P(y_i)\prod\limits_{i=1}^m a_i.$$

## 三.具体流程
（1）从训练集抽取数据，分词、去听用词后，统计不同单词，构建总单词集合，即所有样本（文档）中有效单词的集合；

（2）将单词集合中的单词转换为向量，采取TF-IDF权重策略，构建向量空间模型：

·   文档集合：$D$

   ·   文档数：$|D|$

   ·   词数（Term Count）：$n_{i,j}$，第个$i$样本（文档）内所含第$j$个单词的数量；

   ·   词频（Term Frequency,TF），是对词数的归一化，体现了各单词在某一特定文档的重要性：$$TF_{i,j}=\frac{n_{i,j}}{\sum\limits_{k}n_{k,j}}$$

   ·   逆向文档频率（Inverse Document Frequency,IDF），体现一个单词普遍重要性：$$IDF_i=\log\frac{|D|}{|\{j:t_i\in d_j\}|}$$

   ·   词频逆文档频率（TFIDF），某一特定文件内的高单词频率以及该词语在文档集合中的低文件频率，乘积较大，因此TF\_IDF倾向于过滤掉常见的词语，保留重要词语：
   $$TFIDF_{i,j}=TF_i\times IDF_{i,j}$$

（2）对每个类别计算$P(y_i)$;

（3）对每个特征属性计算所有划分的条件概率$P(a_j|y_i)$；

（4）将测试集文档转换为词向量，并映射到训练集词典，即用训练集词典中的词向量表示测试集文档；

（5）每一个测试集文档$x$，对每个类别计算$P(x|y_i)P(y_i)$;

（5）求$\max\limits_i\{ P(x|y_i)P(y_i)\}$所对应的类别$i$，即是$x$的预测分类.

## 四.代码实例

### 1.创建简单外部数据集

In [ ]:
def loadDataSet():
    postingList=[['my','dog','has','flea','problems','help','please'],
                 ['maybe','not','take','him','to','dog','park','stupid'],
                 ['my','dalmation','is','so','cute','I','love','him','my'],
                 ['stop','posting','stupid','worthless','garbage'],
                 ['mr','licks','ate','my','steak','how','to','stop','him'],
                 ['quit','buying','worthless','dog','food','stupid']]
    classVec=[0,1,0,1,0,1]
    return postingList,classVec

### 2.创建贝叶斯类

（1）初始化：

In [ ]:
class NBayes(object):
    def __init__(self):
        self.vocabulary=[]      # 词典
        self.idf=0              # 词典的IDF权值向量
        self.tf=0               # 训练集的权值矩阵
        self.x_y_prob=0         # P(x|yi)
        self.y_prob={}          # 类别词典 P(yi)
        self.labels=[]          # 
        self.doc_len=0          # 训练集文档数
        self.dict_len=0         # 词典内单词数
        self.testset=0          # 测试集

（2）训练集词向量生成函数：

In [ ]:
    def train_set(self,trainset,classVec):
        self.cate_prob(classVec)                            # 计算P(yi)
        self.doc_len=len(trainset)
        tempset=set()
        [tempset.add(word) for doc in trainset for word in doc] # 生成词典
        self.vocabulary=list(tempset)
        self.dict_len=len(self.vocabulary)
        self.calc_tfidf(trainset)                           # 计算词频数据集
        self.build_x_y_prob()                               # 计算P(x|yi))

（3）cate_prob 函数：计算$P(y_i)$

In [ ]:
    def cate_prob(self,classVec):
        self.labels=classVec
        labeltemps=set(self.labels)
        for labeltemp in labeltemps:
            # 统计列表中的重复分类 self.labels.count(labeltemp)
            self.y_prob[labeltemp]=float(self.labels.count(labeltemp)
                                        )/float(len(self.labels))

（4）calc_tfidf 函数：生成词频向量

In [ ]:
    def calc_tfidf(self,trainset):
        self.idf=np.zeros([1,self.dict_len])            # 1*词典数
        self.tf=np.zeros([self.doc_len,self.dict_len])  # 训练集文档数*词典数
        for index in xrange(self.doc_len):
            for word in trainset[index]:
                self.tf[index,self.vocabulary.index(word)]+=1
            self.tf[index]=self.tf[index]/float(len(trainset[index]))
            for signleword in set(trainset[index]):
                self.idf[0,self.vocabulary.index(signleword)]+=1
        self.idf=np.log(float(self.doc_len)/self.idf)
        self.id=np.multiply(self.tf,self.idf)

（5）build_tdm 函数：构建向量空间$P(x|y_i)$

In [ ]:
    def build_tdm(self):
        self.tdm=np.zeros([len(self.Pcates),self.vocablen]) # 类别行*词典列
        sumlist=np.zeros([len(self.Pcates),1])               # 每个分类总数
        for index in xrange(self.doclength):
            self.tdm[self.labels[index]]+=self.tf[index]
            sumlist[self.labels[index]]=np.sum(self.tdm[self.labels[index]])
        self.tdm=self.tdm/sumlist

（6）map2vocabulary 函数：将测试集的词向量映射到当前词典

In [ ]:
    def map2vocalurary(self,testdata):
        self.testset=np.zeros([1,self.dict_len])
        for word in testdata:
            self.testset[0,self.vocabulary.index(word)]+=1

（7）predict 函数：预测测试文档分类

In [ ]:
    def predict(self,testset):
        # 判断测试文档词典长度与训练集词典长度是否相等
        if np.shape(testset)[1]!=self.dict_len:
            print "输入错误"
            exit(0)
        predvalue=0
        predclass=""
        for x_y_prob_vect,keyclass in zip(self.x_y_prob,self.y_prob):
            temp=np.sum(testset*x_y_prob_vect*self.y_prob[keyclass])
            if temp>predvalue:
                predvalue=temp
                predclass=keyclass
        return predclass

### 3.主程序

In [ ]:
import numpy as np

dataSet,listClasses=loadDataSet()
nb=NBayes()
nb.train_set(dataSet,listClasses)
nb.map2vocabulary(dataSet[0])
print nb.predict(nb.testset)

###### 参考文献：
【1】http://scikit-learn.org/stable/modules/naive_bayes.html

【2】《机器学习算法原理与编程实践》——郑捷